Тема: Стратификация

Видео лекции:  
https://www.youtube.com/watch?v=n6szBbkTYnY
    
Видео семинара:  
https://www.youtube.com/watch?v=x7ynsWEsti8

# Стратифицированный подбор групп

На вход подаются:

    Датафрейм с описанием объектов, которые нужно стратифицировать;
    Название параметров, по которым нужно стратифицировать:
    Размер групп;
    Словарь весов страт (если None, то веса нужно взять пропорционально доле страт в датафрейме);
    Состояние генератора случайных чисел.

Функция должна возвращать пару датафреймов того же формата, что она получила на вход. Доли страт в каждом датафрейме должны в точности совпадать друг с другом и могут немного отличаться от переданных в функцию или посчитанных на основе входных данных, но не более чем на 2 / group_size (предполагается, что размеры страт достаточно велики, чтобы это всегда было реализуемо).

При повторном вызове функции при seed=None полученные группы должны отличаться от вернувшихся при прошлом вызове функции (необходимо использовать рандомизацию).

In [73]:
import numpy as np
import pandas as pd


def select_stratified_groups(data, strat_columns, group_size, weights=None, seed=None):
    """Подбирает стратифицированные группы для эксперимента.

    data - pd.DataFrame, датафрейм с описанием объектов, содержит атрибуты для стратификации.
    strat_columns - List[str], список названий столбцов, по которым нужно стратифицировать.
    group_size - int, размеры групп.
    weights - dict, словарь весов страт {strat: weight}, где strat - либо tuple значений элементов страт,
        например, для strat_columns=['os', 'gender', 'birth_year'] будет ('ios', 'man', 1992),
            либо просто строка/число.
        Если None, определить веса пропорционально доле страт в датафрейме data.
    seed - int, исходное состояние генератора случайных чисел для воспроизводимости
        результатов. Если None, то состояние генератора не устанавливается.

    return (data_pilot, data_control) - два датафрейма того же формата, что и data
        c пилотной и контрольной группами.
    """
    # YOUR_CODE_HERE
    if seed:
        np.random.seed(seed)
        
    # Значения в колонках для стратификации    
    df = data.copy()
    if len(strat_columns) > 1:
        df['strat_values'] = df.apply(lambda x: tuple([x[col] for col in strat_columns]), axis=1)
    else:
        df['strat_values'] = df[strat_columns[0]]
        
    # Генерация weights из исходных данных, если weights не передали
    if not weights:
        weights = df['strat_values'].value_counts(normalize=True)
        weights = dict(weights)
        
    # Число строк в каждой страте
    strat_rows = {strat: round(w * group_size) for strat, w in weights.items()}

    # Сэмплирование
    dfs_a, dfs_b = [], []
    for strat, n_rows in strat_rows.items():
        df_strat = df[df['strat_values'] == strat]
        df_sample = df_strat.sample(2 * n_rows)
        df_sample = df_sample.drop(columns='strat_values')
        dfs_a.append(df_sample.head(df_sample.shape[0] // 2))
        dfs_b.append(df_sample.tail(df_sample.shape[0] // 2))
        
    return pd.concat(dfs_a), pd.concat(dfs_b)

### Проверка

In [74]:
df = pd.DataFrame({'a': [1, 2, 2, 2, 3, 3], 'b': [3, 4, 3, 4, 5, 5], 'c': ['aa', 'aa', 'aa', 'aa', 'cc', 'cc']})
df = pd.concat(100 * [df])

In [75]:
df_a, df_b = select_stratified_groups(df, ['a'], 100)

In [79]:
df['a'].value_counts(normalize=True)

2    0.500000
3    0.333333
1    0.166667
Name: a, dtype: float64

In [80]:
df_a['a'].value_counts(normalize=True)

2    0.50
3    0.33
1    0.17
Name: a, dtype: float64

Доли значений в стратах в исходном и сэмплированном датафрейме совпадают

In [83]:
df_a, df_b = select_stratified_groups(df, ['a', 'b'], 100)
df_a['strat_values'] = df_a.apply(lambda x: tuple([x.a, x.b]), axis=1)
df['strat_values'] = df.apply(lambda x: tuple([x.a, x.b]), axis=1)

In [85]:
df['strat_values'].value_counts(normalize=True)

(2, 4)    0.333333
(3, 5)    0.333333
(1, 3)    0.166667
(2, 3)    0.166667
Name: strat_values, dtype: float64

In [84]:
df_a['strat_values'].value_counts(normalize=True)

(2, 4)    0.33
(3, 5)    0.33
(1, 3)    0.17
(2, 3)    0.17
Name: strat_values, dtype: float64

При стратификации по нескольким колонкам доли страт тоже совпадают.